In [163]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
import numpy as np
import pickle

In [1]:
# a = pickle.load(open("singapore/shells/Ang Mo Kio.p", "rb"))
# a

In [45]:
sing_df = pd.read_excel("sheets/land_use_constrained_data.xlsx", sheet_name=1, header=5, usecols="B:V", nrows=9)

sf_df = pd.read_excel("sheets/land_use_constrained_data.xlsx", sheet_name=1, header=55, usecols="B:V", nrows=11)

sing_df

,Planning zone,# of Zone Types,% Open Space,% Residential,% Commercial,% Industrial,% Mixed Use,% Paved Area,Distance covered in 30 minutes using Public Transportation (km),Distance covered in 30 minutes using a Car (km),...,Distance covered in 30 minutes Cycling (km),"Cost of daily commute per capita, per km using bus (S$)",Cost of daily commute per capita per km using MRT (S$),Cost of daily commute per capita per km Driving (S$),% households with income >= USD8000/month,% residents with high school degrees,% residents with university degrees,% residents employed,% residents 15y + literate,% residents 15y + literate in 2+ languages
0,Bukit Merah,5,0.387,0.283,0.121,0.021,0.063,0.126,6.13,14.66,...,6.84,0.13,0.06,0.30,0.4105,0.6912,0.3211,0.6121,0.9546,0.6615
1,Bukit Timah,5,0.030,0.655,0.004,0.003,0.156,0.152,1.02,15.41,...,4.94,0.13,0.06,0.30,0.6971,0.9217,0.6633,0.5877,0.9912,0.7409
2,Geylang,5,0.063,0.416,0.233,0.023,0.073,0.193,4.16,22.02,...,6.11,0.13,0.06,0.29,0.4081,0.6993,0.3013,0.6311,0.9540,0.6697
3,Kallang,5,0.285,0.288,0.126,0.027,0.063,0.211,5.96,19.98,...,6.17,0.13,0.06,0.30,0.4116,0.6888,0.3311,0.6217,0.9513,0.6481
4,Novena,5,0.118,0.579,0.054,0.022,0.059,0.168,10.74,21.14,...,6.41,0.13,0.06,0.30,0.5478,0.8130,0.4786,0.6261,0.9740,0.7019
5,Queenstown,5,0.115,0.454,0.156,0.045,0.072,0.159,11.36,12.65,...,7.32,0.12,0.06,0.29,0.4482,0.7317,0.3735,0.6015,0.9627,0.6721
6,Tanglin,5,0.191,0.621,0.030,0.016,0.030,0.112,4.63,18.58,...,6.34,0.15,0.08,0.32,0.7249,0.9507,0.7377,0.6097,0.9949,0.7162
7,Toa Payoh,5,0.077,0.457,0.153,0.001,0.109,0.203,5.62,21.01,...,6.71,0.12,0.06,0.29,0.4084,0.7269,0.3196,0.6065,0.9584,0.6700
8,CBD Area,5,0.319,0.130,0.190,0.040,0.126,0.193,6.23,18.32,...,5.23,0.16,0.08,0.32,0.5858,0.8288,0.5620,0.6507,0.9742,0.7050


In [60]:
sing_df.iloc[:, 8:15]

,Distance covered in 30 minutes using Public Transportation (km),Distance covered in 30 minutes using a Car (km),Distance covered in 30 minutes Walking (km),Distance covered in 30 minutes Cycling (km),"Cost of daily commute per capita, per km using bus (S$)",Cost of daily commute per capita per km using MRT (S$),Cost of daily commute per capita per km Driving (S$)
0,6.13,14.66,2.18,6.84,0.13,0.06,0.30
1,1.02,15.41,2.04,4.94,0.13,0.06,0.30
2,4.16,22.02,1.95,6.11,0.13,0.06,0.29
3,5.96,19.98,2.02,6.17,0.13,0.06,0.30
4,10.74,21.14,2.16,6.41,0.13,0.06,0.30
5,11.36,12.65,2.17,7.32,0.12,0.06,0.29
6,4.63,18.58,2.17,6.34,0.15,0.08,0.32
7,5.62,21.01,1.80,6.71,0.12,0.06,0.29
8,6.23,18.32,2.19,5.23,0.16,0.08,0.32


In [159]:
def land_use_metric(arr):
    """arr should be the land use metrics that are not paved area"""
    # goal: even split between the 5 land use types, reserving 10% of area for paved areas 
    goal_ratio = 0.9/4
    ratios = [i - goal_ratio for i in arr]
    return np.sum(ratios)

def make_metrics(df, city):
    # land use 
    df["Sum Land Use"] = df.iloc[:, 2:8].sum(1)
    adjusted_land_use_df = df.iloc[:, 2:8].apply(lambda x: x/df["Sum Land Use"])
    land_temp = adjusted_land_use_df.iloc[:,0:4].apply(lambda x: land_use_metric(x), axis=1)
    land_metric = 1 - land_temp*-1
    diversity_metric = pd.Series(land_metric, name="Land Use Diversity")

    # transport 
    transport_metric = df.iloc[:, 8]
    a = pd.cut(transport_metric, bins=4, retbins=False)
    transport_metric = a.apply(lambda x: x.right)

    # qol 
    qol_metric = df.iloc[:, -6]

    city_entry = 0.1 # sf 
    if city == "singapore":
        city_entry = 0.2
    

    df2 = pd.DataFrame([diversity_metric, transport_metric, qol_metric, pd.Series([city_entry]*len(qol_metric))], index=["land", "transport", "education", "city"]).T

    return df2.round(1)

In [160]:
sing_dfa = make_metrics(sing_df, "singapore")
sf_dfa = make_metrics(sing_df, "san francisco")
dfa = pd.concat([sf_dfa, sing_dfa], ignore_index=True)
dfa

,land,transport,education,city
0,0.9,6.2,0.7,0.1
1,0.8,3.6,0.9,0.1
2,0.8,6.2,0.7,0.1
3,0.8,6.2,0.7,0.1
4,0.9,11.4,0.8,0.1
5,0.9,11.4,0.7,0.1
6,1.0,6.2,1.0,0.1
7,0.8,6.2,0.7,0.1
8,0.8,8.8,0.8,0.1
9,0.9,6.2,0.7,0.2


In [133]:
model = ols("""education ~ C(city)""", data=dfa).fit()
model.summary()

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.063
Method:                 Least Squares   F-statistic:                -5.521e-15
Date:                Wed, 15 Mar 2023   Prob (F-statistic):               1.00
Time:                        07:06:02   Log-Likelihood:                 16.916
No. Observations:                  18   AIC:                            -29.83
Df Residuals:                      16   BIC:                            -28.05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.7836      0.033     23.442      0.000       0.713       0.854
C(city)[T.0.2]  6.428e-17      0.047   1.36e-15      1.000      -0.100       0.100
==============================================================================
Omnibus:                        3.361   Durbin-Watson:                   3.016
Prob(Omnibus):                  0.186   Jarque-Bera (JB):                2.124
Skew:                           0.640   Prob(JB):                        0.346
Kurtosis:                       1.906   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [149]:
model = ols("""education ~ C(city) + C(land) + C(city):C(land) """, data=dfa).fit()

model.summary()

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.427
Method:                 Least Squares   F-statistic:                     3.531
Date:                Wed, 15 Mar 2023   Prob (F-statistic):             0.0341
Time:                        07:15:27   Log-Likelihood:                 23.509
No. Observations:                  18   AIC:                            -35.02
Df Residuals:                      12   BIC:                            -29.68
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         0.7600      0.036     21.169      0.000       0.682       0.838
C(city)[T.0.2]                 3.919e-16      0.051   7.72e-15      1.000      -0.111       0.111
C(land)[T.0.9]                   -0.0267      0.059     -0.455      0.657      -0.154       0.101
C(land)[T.1.0]                    0.2400      0.088      2.729      0.018       0.048       0.432
C(city)[T.0.2]:C(land)[T.0.9]  5.773e-17      0.083   6.96e-16      1.000      -0.181       0.181
C(city)[T.0.2]:C(land)[T.1.0]  2.853e-16      0.124   2.29e-15      1.000      -0.271       0.271
==============================================================================
Omnibus:                        3.663   Durbin-Watson:                   2.935
Prob(Omnibus):                  0.160   Jarque-Bera (JB):                2.720
Skew:                           0.940   Prob(JB):                        0.257
Kurtosis:                       2.696   Cond. No.                         9.64
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [161]:
model = ols("""education ~ C(city) + C(transport) + C(land) +
               C(city):C(transport) + C(city):C(land) + C(transport):C(land) +
               C(city):C(transport):C(land)""", data=dfa).fit()

model.summary()

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     9.879
Date:                Wed, 15 Mar 2023   Prob (F-statistic):            0.00535
Time:                        07:21:37   Log-Likelihood:                 41.919
No. Observations:                  18   AIC:                            -59.84
Df Residuals:                       6   BIC:                            -49.15
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                              0.9000      0.041     22.045      0.000       0.800       1.000
C(city)[T.0.2]                                      3.628e-16      0.058   6.28e-15      1.000      -0.141       0.141
C(transport)[T.6.2]                                   -0.2000      0.047     -4.243      0.005      -0.315      -0.085
C(transport)[T.8.8]                                   -0.1000      0.058     -1.732      0.134      -0.241       0.041
C(transport)[T.11.4]                                  -0.0600      0.022     -2.714      0.035      -0.114      -0.006
C(land)[T.0.9]                                        -0.0300      0.021     -1.406      0.209      -0.082       0.022
C(land)[T.1.0]                                         0.1500      0.024      6.364      0.001       0.092       0.208
C(city)[T.0.2]:C(transport)[T.6.2]                 -3.322e-16      0.067  -4.98e-15      1.000      -0.163       0.163
C(city)[T.0.2]:C(transport)[T.8.8]                 -5.551e-16      0.082   -6.8e-15      1.000      -0.200       0.200
C(city)[T.0.2]:C(transport)[T.11.4]                -9.079e-17      0.031   -2.9e-15      1.000      -0.077       0.077
C(city)[T.0.2]:C(land)[T.0.9]                      -1.924e-16      0.030  -6.38e-15      1.000      -0.074       0.074
C(city)[T.0.2]:C(land)[T.1.0]                      -1.308e-16      0.033  -3.92e-15      1.000      -0.082       0.082
C(transport)[T.6.2]:C(land)[T.0.9]                     0.0300      0.030      1.000      0.356      -0.043       0.103
C(transport)[T.8.8]:C(land)[T.0.9]                          0          0        nan        nan           0           0
C(transport)[T.11.4]:C(land)[T.0.9]                   -0.0600      0.022     -2.714      0.035      -0.114      -0.006
C(transport)[T.6.2]:C(land)[T.1.0]                     0.1500      0.024      6.364      0.001       0.092       0.208
C(transport)[T.8.8]:C(land)[T.1.0]                          0          0        nan        nan           0           0
C(transport)[T.11.4]:C(land)[T.1.0]                         0          0        nan        nan           0           0
C(city)[T.0.2]:C(transport)[T.6.2]:C(land)[T.0.9]  -3.363e-16      0.042  -7.93e-15      1.000      -0.104       0.104
C(city)[T.0.2]:C(transport)[T.8.8]:C(land)[T.0.9]           0          0        nan        nan           0           0
C(city)[T.0.2]:C(transport)[T.11.4]:C(land)[T.0.9] -2.946e-17      0.031  -9.42e-16      1.000      -0.077       0.077
C(city)[T.0.2]:C(transport)[T.6.2]:C(land)[T.1.0]   -1.78e-16      0.033  -5.34e-15      1.000      -0.082       0.082
C(city)[T.0.2]:C(transport)[T.8.8]:C(land)[T.1.0]           0          0        nan        nan           0  

In [162]:
sm.stats.anova_lm(model, typ=2)

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/statsmodels/base/model.py:1900: RuntimeWarning: invalid value encountered in scalar divide
  F /= J
/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 0
  warnings.warn('covariance of 

,sum_sq,df,F,PR(>F)
C(city),NaN,1.0,NaN,NaN
C(transport),1.361099e-02,3.0,2.722197e+00,0.150055
C(land),NaN,2.0,NaN,NaN
C(city):C(transport),7.453171e-32,3.0,1.490634e-29,1.000000
C(city):C(land),NaN,2.0,NaN,NaN
C(transport):C(land),3.480000e-01,6.0,3.480000e+01,0.000344
C(city):C(transport):C(land),5.160386e-31,6.0,5.160386e-29,1.000000
Residual,1.000000e-02,6.0,NaN,NaN


## Example

In [3]:
#create DataFrame
df = pd.DataFrame({'program': np.repeat([1, 2], 20),
                   'gender': np.tile(np.repeat(['M', 'F'], 10), 2),
                   'division': np.tile(np.repeat([1, 2], 5), 4),
                   'height': [7, 7, 8, 8, 7, 6, 6, 5, 6, 5,
                              5, 5, 4, 5, 4, 3, 3, 4, 3, 3,
                              6, 6, 5, 4, 5, 4, 5, 4, 4, 3,
                              2, 2, 1, 4, 4, 2, 1, 1, 2, 1]})

In [6]:
df.head()

,program,gender,division,height
0,1,M,1,7
1,1,M,1,7
2,1,M,1,8
3,1,M,1,8
4,1,M,1,7


In [5]:
model = ols("""height ~ C(program) + C(gender) + C(division) +
               C(program):C(gender) + C(program):C(division) + C(gender):C(division) +
               C(program):C(gender):C(division)""", data=df).fit()

sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(program),3.610000e+01,1.0,6.563636e+01,2.983934e-09
C(gender),6.760000e+01,1.0,1.229091e+02,1.714432e-12
C(division),1.960000e+01,1.0,3.563636e+01,1.185218e-06
C(program):C(gender),4.718528e-29,1.0,8.579142e-29,1.000000e+00
C(program):C(division),4.000000e-01,1.0,7.272727e-01,4.001069e-01
C(gender):C(division),1.000000e-01,1.0,1.818182e-01,6.726702e-01
C(program):C(gender):C(division),1.000000e-01,1.0,1.818182e-01,6.726702e-01
Residual,1.760000e+01,32.0,NaN,NaN


In [ ]:
# get data 
# do two way anova 